In [10]:
from datastreams.datastream import DataStream

import pandas as pd

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [2]:
endpointList = [
    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum', 
    'https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum',
    'https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum',
    'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2', # https://thegraph.com/hosted-service/subgraph/uniswap/uniswap-v2 # NON-MESSARi SUBGRAPH
    'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    'https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum'
    ]


In [3]:
ds = DataStream(endpointList)

# endpoints loaded into DataStream
print(f' the endpoints are: {ds.streamerDict.keys()}')

# check the keys for a single Streamer "sushiswap-ethereum"
# print(ds.streamerDict['sushiswap-ethereum'].queryDict.keys())


 the endpoints are: dict_keys(['sushiswap-ethereum', 'balancer-v2-ethereum', 'curve-finance-ethereum', 'uniswap-v2', 'uniswap-v3-ethereum', 'saddle-finance-ethereum'])


In [4]:
common_keys = ds.getCommonQueryKeys()
print(f'common_keys: {common_keys}') # the common_keys are very small because uniswap-v2 is not a messari standardized subgraph.

common_keys: ['swaps', 'tokens']


In [5]:
# make a new streamerDict excluding uniswap-v2
new_dict = ds.streamerDict
new_dict.pop('uniswap-v2')
new_common_keys = ds.getCommonQueryKeys(streamerList=new_dict)
print(f'new_common_keys: {new_common_keys}') # we now have a much larger list of common keys after dropping uniswap-v2

new_common_keys: ['usageMetricsHourlySnapshots', 'liquidityPoolDailySnapshots', 'swaps', 'withdraws', 'financialsDailySnapshots', 'usageMetricsDailySnapshots', 'liquidityPoolFees', 'tokens', 'events', 'protocols', 'activeAccounts', 'liquidityPools', 'rewardTokens', 'deposits', 'liquidityPoolHourlySnapshots', 'dexAmmProtocols', 'accounts']


In [6]:
tokens_dfs = ds.querySubgraphs(new_dict, 'tokens', query_size=77)

querying https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum for tokens...
FIELD - tokens
querying https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum for tokens...
FIELD - tokens
querying https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum for tokens...
FIELD - tokens
querying https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum for tokens...
FIELD - tokens
querying https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum for tokens...
FIELD - tokens


In [11]:
# concat dfs in token_dfs into a single dataframe
tokens_df = pd.concat(tokens_dfs, axis=0, ignore_index=True)

In [14]:
tokens_df

,tokens_id,tokens_name,tokens_symbol,tokens_decimals,tokens_lastPriceUSD,tokens_lastPriceBlockNumber,endpoint,tokens__pool_id
0,0x0000000000004946c0e9f43f4dee607b0ef1fa1c,Chi Gastoken by 1inch,CHI,0,0.000000,0.0,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN
1,0x0000000000085d4780b73119b644ae5ecd22b376,TrueUSD,TUSD,18,0.998081,0.0,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN
2,0x0000000000095413afc295d19edeb1ad7b71c952,Tokenlon,LON,18,0.594239,0.0,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN
3,0x00000006e55a9364b657e3b91cd0411b4fd11ac2,Adidas Originals Metaverse,ADIDAS,18,0.000000,0.0,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN
4,0x0000a1c00009a619684135b824ba02f7fbf3a572,Alchemy,ALCH,18,0.000000,0.0,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN
...,...,...,...,...,...,...,...,...
360,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,renBTC,renBTC,8,21096.002120,16445596.0,https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum,None
361,0xecd7790fd35e5f529a63cf3e058b555f2ff23234,veFRAX-FRAXBP Earn Pool,veFRAX-FRAXBP,18,1.004914,16439220.0,https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum,None
362,0xf1dc500fde233a4055e25e5bbf516372bc4f6871,Saddle DAO,SDL,18,NaN,NaN,https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum,None
363,0xf32e91464ca18fc156ab97a697d6f8ae66cd21a3,Saddle WBTC/renBTC/sBTC,saddleWRenSBTC,18,NaN,NaN,https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum,0xdf3309771d2bf82cb2b6c56f9f5365c8bd97c4f2


In [8]:
pools_dfs = ds.querySubgraphs(new_dict, 'liquidityPools', query_size=77)

querying https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum for liquidityPools...
FIELD - liquidityPools
querying https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum for liquidityPools...
FIELD - liquidityPools
querying https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum for liquidityPools...
FIELD - liquidityPools
querying https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum for liquidityPools...
FIELD - liquidityPools
querying https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum for liquidityPools...
FIELD - liquidityPools


In [15]:
# concat dfs in pools_dfs into a single dataframe
pools_df = pd.concat(pools_dfs, axis=0, ignore_index=True)

In [16]:
pools_df

,liquidityPools_id,liquidityPools_protocol_id,liquidityPools_name,liquidityPools_symbol,liquidityPools_outputToken_id,liquidityPools_isSingleSided,liquidityPools_createdTimestamp,liquidityPools_createdBlockNumber,liquidityPools_totalValueLockedUSD,liquidityPools_cumulativeSupplySideRevenueUSD,liquidityPools_cumulativeProtocolSideRevenueUSD,liquidityPools_cumulativeTotalRevenueUSD,liquidityPools_cumulativeVolumeUSD,liquidityPools_outputTokenSupply,liquidityPools_outputTokenPriceUSD,liquidityPools_stakedOutputTokenAmount,endpoint,liquidityPools__poolId,liquidityPools__registryAddress,liquidityPools__gaugeAddress,liquidityPools__basePool_id
0,0x00040a7ebfc9f6fbce4d23bd66b79a603ba1c323,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,SushiSwap TrueUSD/Stable Yield Credit,TrueUSD/Stable Yield Credit,0x00040a7ebfc9f6fbce4d23bd66b79a603ba1c323,False,1609668213,11580839,7.720463e-09,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0,0.000000,NaN,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN,NaN,NaN,NaN
1,0x00088e1f7510370fab86a7bd10578b578c61c723,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,SushiSwap Tokenlon/SushiToken,Tokenlon/SushiToken,0x00088e1f7510370fab86a7bd10578b578c61c723,False,1617788940,12191971,7.690761e-01,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,38917430505825711,19.761739,NaN,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN,NaN,NaN,NaN
2,0x0013df4ec514d5839ad4f535e4987498ebb58bf5,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,SushiSwap Orange Side Punk #7973/Wrapped Ether,Orange Side Punk #7973/Wrapped Ether,0x0013df4ec514d5839ad4f535e4987498ebb58bf5,False,1630465727,13137302,8.606310e+00,2.704262e+00,0.540852,3.245114e+00,1.081705e+03,4878579919686138,1764.101454,NaN,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN,NaN,NaN,NaN
3,0x0018fb451a46ae397b8569936bc5bb5ff03cfd18,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,SushiSwap Bella/Wrapped Ether,Bella/Wrapped Ether,0x0018fb451a46ae397b8569936bc5bb5ff03cfd18,False,1622454793,12541436,4.656667e+01,1.762791e+04,3525.582587,2.115350e+04,7.051165e+06,1892056239385899267,24.611676,NaN,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN,NaN,NaN,NaN
4,0x001b6450083e531a5a7bf310bd2c1af4247e23d4,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,SushiSwap UMA Voting Token v1/Wrapped Ether,UMA Voting Token v1/Wrapped Ether,0x001b6450083e531a5a7bf310bd2c1af4247e23d4,False,1599677178,10829243,2.798018e+05,1.309587e+06,261917.339838,1.571504e+06,5.238347e+08,1894856357422829838213,147.663836,NaN,https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,0xdf3309771d2bf82cb2b6c56f9f5365c8bd97c4f2,0x5bdb37d0ddea3a90f233c7b7f6b9394b6b2eef34,Saddle WBTC/renBTC/sBTC,saddleWRenSBTC,0xf32e91464ca18fc156ab97a697d6f8ae66cd21a3,False,1629751863,13083826,1.920121e+05,5.503520e+04,31968.911340,8.700411e+04,2.174844e+08,11347920071480747754,16920.467921,4.539457e+15,https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum,NaN,NaN,NaN,None
335,0xe41389921cc14e2159232bd6debc78924307e5a9,0x5bdb37d0ddea3a90f233c7b7f6b9394b6b2eef34,Saddle saddle-busd LP Token,saddle-busd,0xdd0e2d8f8e463dd7d54e26b3c38bb26d849eb662,False,1664915555,15677173,6.814496e+00,2.550660e-04,0.000255,5.101320e-04,1.306115e+00,9999891196796552806,0.681457,NaN,https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum,NaN,NaN,NaN,0x13cc34aa8037f722405285ad2c82fe570bfa2bdc
336,0xf74ebe6e5586275dc4ced78f5dbef31b1efbe7a5,0x5bdb37d0ddea3a90f233c7b7f6b9394b6b2eef34,Saddle tBTCv2/saddleWRenSBTC,saddletBTC,0x122eca07139eb368245a29fb702c9ff11e9693b7,False,1630370737,13130191,2.112218e+04,5.949456e+03,0.000000,5.949456e+03,1.485938e+07,1257249194915947826,16800.316065,NaN,https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum,NaN,NaN,NaN,0xdf3309771d2bf82cb2b6c56f9f5365c8bd97c4f2
337,0xfa9ed0309bf79eb84c847819